# Comparing randomized search and grid search for hyperparameter estimation

http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html#example-model-selection-randomized-search-py

In [ ]:
"""
=========================================================================
Comparing randomized search and grid search for hyperparameter estimation
=========================================================================

Compare randomized search and grid search for optimizing hyperparameters of a
random forest.
All parameters that influence the learning are searched simultaneously
(except for the number of estimators, which poses a time / quality tradeoff).

The randomized search and the grid search explore exactly the same space of
parameters. The result in parameter settings is quite similar, while the run
time for randomized search is drastically lower.

The performance is slightly worse for the randomized search, though this
is most likely a noise effect and would not carry over to a held-out test set.

Note that in practice, one would not search over this many different parameters
simultaneously using grid search, but pick only the ones deemed most important.
"""

In [7]:
import numpy as np

from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

from pandas import DataFrame as DF
import tak.tak as tw

In [2]:
# get some data
iris = load_digits()
X, y = iris.data, iris.target

# build a classifier
clf = RandomForestClassifier(n_estimators=20)

In [4]:
clf_prefit = clf

In [5]:
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [9]:
# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
DF(param_dist).T

,0,1
bootstrap,True,False
criterion,gini,entropy
max_depth,3,NaN
max_features,<scipy.stats._distn_infrastructure.rv_frozen o...,<scipy.stats._distn_infrastructure.rv_frozen o...
min_samples_leaf,<scipy.stats._distn_infrastructure.rv_frozen o...,<scipy.stats._distn_infrastructure.rv_frozen o...
min_samples_split,<scipy.stats._distn_infrastructure.rv_frozen o...,<scipy.stats._distn_infrastructure.rv_frozen o...


In [10]:
# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

RandomizedSearchCV took 1.67 seconds for 20 candidates parameter settings.


In [11]:
report(random_search.grid_scores_)

Model with rank: 1
Mean validation score: 0.923 (std: 0.007)
Parameters: {'bootstrap': False, 'min_samples_leaf': 2, 'min_samples_split': 8, 'criterion': 'gini', 'max_features': 4, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.911 (std: 0.006)
Parameters: {'bootstrap': False, 'min_samples_leaf': 9, 'min_samples_split': 4, 'criterion': 'entropy', 'max_features': 5, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.905 (std: 0.009)
Parameters: {'bootstrap': False, 'min_samples_leaf': 2, 'min_samples_split': 3, 'criterion': 'gini', 'max_features': 1, 'max_depth': None}



In [12]:
# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.grid_scores_)))
report(grid_search.grid_scores_)

GridSearchCV took 18.00 seconds for 216 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.934 (std: 0.014)
Parameters: {'bootstrap': False, 'min_samples_leaf': 3, 'min_samples_split': 1, 'criterion': 'entropy', 'max_features': 10, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.932 (std: 0.016)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 3, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.932 (std: 0.014)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 10, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}

